# Creating Squeak to Speaks database
This notebook is single-use, serving only to initially create the SQL database to be used

In [1]:
import sqlite3
import pandas as pd

## Creating the db

In [2]:
def create_database():
    connection = sqlite3.connect("Squeaktospeak_db.db")
    cursor = connection.cursor()

    # Create Users table
    cursor.execute('''
    CREATE TABLE Users (
        user_id INTEGER PRIMARY KEY,
        username VARCHAR(30),
        password VARCHAR(50),
        email NVARCHAR(50),
        country NVARCHAR(30)
    );
    ''')

    # Create Mood_tracker table
    cursor.execute('''
    CREATE TABLE Mood_tracker (
        mood_id INTEGER PRIMARY KEY,
        user_id INTEGER,
        mood NVARCHAR(20),
        date DATETIME,
        FOREIGN KEY (user_id) REFERENCES Users(user_id),
        UNIQUE(user_id, date)           
    );
    ''')

    # Create Journal table
    cursor.execute('''
    CREATE TABLE Journal (
        message_id INTEGER PRIMARY KEY,
        user_id INTEGER,
        message TEXT,
        date DATETIME,
        hide_yn BIT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id),
        UNIQUE(user_id, date)
    );
    ''')

    # Create Gratitude_entries table
    cursor.execute('''
    CREATE TABLE Gratitude_entries (
        gratitude_id INTEGER PRIMARY KEY,
        date DATETIME,
        comment NVARCHAR(200)
    );
    ''')

    # Create Helpful_Info table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Helpful_Info (
        info_id INTEGER PRIMARY KEY,
        TYPE NVARCHAR(255),
        name NVARCHAR(100),
        country NVARCHAR(30),
        email NVARCHAR(50),
        website NVARCHAR(80),
        phone NVARCHAR(12),
        organization NVARCHAR(255)
    );
    ''')

# Create Help_lines table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Help_lines (
        info_id INTEGER PRIMARY KEY,
        TYPE NVARCHAR(255),
        name NVARCHAR(100),
        country NVARCHAR(30),
        email NVARCHAR(50),
        website NVARCHAR(80),
        phone NVARCHAR(12),
        organization NVARCHAR(255),
        always_open BIT,
        specialty NVARCHAR(100)
    );
    ''')

    # Create Therapists table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Therapists (
        info_id INTEGER PRIMARY KEY,
        TYPE NVARCHAR(255),
        name NVARCHAR(100),
        country NVARCHAR(30),
        email NVARCHAR(50),
        website NVARCHAR(80),
        phone NVARCHAR(12),
        organization NVARCHAR(255),
        always_open BIT,
        location NVARCHAR(100),
        avg_consult_price SMALLINT,
        specialty NVARCHAR(100),
        online_option BIT,
        in_person_option BIT
    );
    ''')

    # Create Support_groups table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Support_groups (
        info_id INTEGER PRIMARY KEY,
        TYPE NVARCHAR(255),
        name NVARCHAR(100),
        country NVARCHAR(30),
        email NVARCHAR(50),
        website NVARCHAR(80),
        phone NVARCHAR(12),
        organization NVARCHAR(255),
        session_price SMALLINT,
        target_audience NVARCHAR(100),
        location NVARCHAR(80)
    );
    ''')


    connection.commit()
    return connection




## Populating the database

In [3]:
def populate_database(connection):
    cursor = connection.cursor()

    # Load data from Excel
    file_path = "squeaktospeak_db.xlsx"  # Update path if necessary
    excel_data = pd.ExcelFile(file_path)
    data = {sheet: excel_data.parse(sheet) for sheet in excel_data.sheet_names}

    # Insert data into Users table
    users = data['users']
    users.to_sql('Users', connection, if_exists='append', index=False)

    # Insert data into Helpful_Info table
    helpful_info = data['helpful_info']
    helpful_info.to_sql('Helpful_Info', connection, if_exists='append', index=False)

    # Insert data into Help_lines table
    help_lines = data['help_lines']
    help_lines.to_sql('Help_lines', connection, if_exists='append', index=False)

    # Insert data into Therapists table
    therapists = data['therapists']
    therapists.to_sql('Therapists', connection, if_exists='append', index=False)

    # Insert data into Support_groups table
    support_groups = data['support_groups']
    support_groups.to_sql('Support_groups', connection, if_exists='append', index=False)

    # Insert data into Journal table
    journal = data['journal']
    for index, row in journal.iterrows():
        user_id, date = row['user_id'], row['date']
        cursor.execute(
            '''
            SELECT COUNT(*) FROM Journal WHERE user_id = ? AND date = ?
            ''',
            (user_id, date),
        )
        if cursor.fetchone()[0] == 0:  # No duplicate
            cursor.execute(
                '''
                INSERT INTO Journal (message_id, user_id, message, date, hide_yn)
                VALUES (?, ?, ?, ?, ?)
                ''',
                (row['message_id'], user_id, row['message'], date, row['hide_yn']),
            )
        else:
            print(f"Skipping duplicate journal entry for user_id={user_id} on date={date}")




    # Insert data into Mood_tracker table
    mood_tracker = data['mood_tracker']
    for index, row in mood_tracker.iterrows():
        user_id, date = row['user_id'], row['Date']
        cursor.execute(
            '''
            SELECT COUNT(*) FROM Mood_tracker WHERE user_id = ? AND date = ?
            ''',
            (user_id, date),
        )
        if cursor.fetchone()[0] == 0:  # No duplicate
            cursor.execute(
                '''
                INSERT INTO Mood_tracker (mood_id, user_id, mood, date)
                VALUES (?, ?, ?, ?)
                ''',
                (row['mood_id'], user_id, row['mood'], date),
            )
        else:
            print(f"Skipping duplicate entry for user_id={user_id} on date={date}")

    connection.commit()


    # Insert data into Gratitude_entries table
    gratitude = data['gratitude']
    gratitude.to_sql('Gratitude_entries', connection, if_exists='append', index=False)



In [4]:
connection = create_database()
populate_database(connection)
connection.close()

Skipping duplicate entry for user_id=12 on date=2023-01-04
Skipping duplicate entry for user_id=13 on date=2023-01-06
Skipping duplicate entry for user_id=32 on date=2023-01-08
Skipping duplicate entry for user_id=40 on date=2023-01-08
Skipping duplicate entry for user_id=43 on date=2023-01-06
